# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением F1-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте F1-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте AUC-ROC, сравнивайте её значение с F1-мерой.

### Подключим необходимые библиотеки

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn import tree
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import train_test_split 
from sklearn.metrics import (
    accuracy_score, 
    roc_auc_score, 
    f1_score, 
    recall_score
)
from sklearn.preprocessing import StandardScaler 
from sklearn.utils import shuffle

## Откройте и изучите файл

### Откроем файл и посмотрим на первые строки

In [2]:
try:
    df = pd.read_csv('')
except:
    df = pd.read_csv('')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [3]:
print(df.shape)
df.head()

(10000, 14)


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


<u>Признаки:</u>
- <b>RowNumber</b> — индекс строки в данных
- <b>CustomerId</b> — уникальный идентификатор клиента
- <b>Surname</b> — фамилия
- <b>CreditScore</b> — кредитный рейтинг
- <b>Geography</b> — страна проживания
- <b>Gender</b> — пол
- <b>Age</b> — возраст
- <b>Tenure</b> — сколько лет человек является клиентом банка
- <b>Balance</b> — баланс на счёте
- <b>NumOfProducts</b> — количество продуктов банка, используемых клиентом
- <b>HasCrCard</b> — наличие кредитной карты
- <b>IsActiveMember</b> — активность клиента
- <b>EstimatedSalary</b> — предполагаемая зарплата

<u>Целевой признак:</u>
- <b>Exited</b> — факт ухода клиента

### Посмотрим наличие пропусков и обработаем их

In [4]:
df.isna().sum() #найдем тустые ячейки

RowNumber            0
CustomerId           0
Surname              0
CreditScore          0
Geography            0
Gender               0
Age                  0
Tenure             909
Balance              0
NumOfProducts        0
HasCrCard            0
IsActiveMember       0
EstimatedSalary      0
Exited               0
dtype: int64

In [5]:
#df['Tenure'] = df['Tenure'].fillna(df['Tenure'].median()) 
df = df.dropna(subset=['Tenure']) 
#заполним пустые ячейки медианным значением

In [6]:
df.isna().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

### Посмотрим наличие дубликатов

In [7]:
df.duplicated().sum()

0

Поскольку перед моделью стоит задача классификации (ушел клиент или нет), то для нас не имеют значения данные из колонок <b>RowNumber, CustomerId, Surname</b> - удалим их.

### Удалим ненужные колонки

In [8]:
df = df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

### Посмотрим снова на данные после обрабоки

In [9]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


## Разбейте данные на выборки

In [10]:
features = df.drop(['Exited'], axis=1)
target = df['Exited']

features_nf, features_valid, target_nf, target_valid = train_test_split( 
    features, target, test_size=0.2, random_state=13)#в принципе, я думаю, можно было перезаписать признаки и цели

features_train, features_test, target_train, target_test = train_test_split(
    features_nf, target_nf, test_size=0.25, random_state=13)

print('Количество строк в валидационной выборке:', target_valid.count(),
      ', что составляет', target_valid.count()/target.count())
print('Количество строк в тестовой выборке:', target_test.count(),
      ', что составляет', target_test.count()/target.count())
print('Количество строк в тренировочной выборке:', target_train.count(),
      ', что составляет', target_train.count()/target.count())

Количество строк в валидационной выборке: 1819 , что составляет 0.2000879991200088
Количество строк в тестовой выборке: 1818 , что составляет 0.1999780002199978
Количество строк в тренировочной выборке: 5454 , что составляет 0.5999340006599934


In [11]:
features_train = pd.get_dummies(features_train, drop_first=True)
features_valid = pd.get_dummies(features_valid, drop_first=True)
features_test = pd.get_dummies(features_test, drop_first=True)

## Исследуйте модели

In [12]:
pd.options.mode.chained_assignment = None
#отключим тонну красного текста сообщающего неихвестно о чем...

In [13]:
numeric = ['CreditScore', 'Age', 'Tenure', 'Balance', 'EstimatedSalary']

scaler = StandardScaler()
scaler.fit(features_train[numeric])

features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])

### Посмотрим на поведедение различных моделей

In [14]:
#Посмотрим модель решающего дерева
best_model = None
best_result = 0
train_list = []
valid_list = []
for depth in range(1, 20):
    model = DecisionTreeClassifier(random_state=12345,
                                   max_depth=depth) #модель с заданной глубиной дерева
    model.fit(features_train, target_train) # обучение модели
    predictions = model.predict(features_valid) #предсказания модели
    result = f1_score(target_valid, predictions) 
    acc = accuracy_score(target_valid, predictions)
    valid_list.append(result)
    if result > best_result:
        best_model = model
        best_result = result
        best_depth = depth
        best_acc = acc
        probabilities_valid = best_model.predict_proba(features_valid)
        probabilities_one_valid = probabilities_valid[:, 1]
        auc_roc = roc_auc_score(target_valid, probabilities_one_valid)

In [15]:
print("Наилучшая глубина: ", best_depth)
print("Accuracy лучшей модели:", best_acc)
print("F1 для Дерева решений:", best_result)
print("AUC-ROC для Дерева решени:", auc_roc)

Наилучшая глубина:  7
Accuracy лучшей модели: 0.8614623419461243
F1 для Дерева решений: 0.5594405594405595
AUC-ROC для Дерева решени: 0.8328027147694245


Целевая F1 должна быть равна 0.59, наша 0.56. Значит проверим другую модель.

In [16]:
#Модель логистической регрессии
model_log = LogisticRegression(random_state=12345, 
                               solver='liblinear', 
                               max_iter=100)
model_log.fit(features_train, target_train)
predictions_log = model_log.predict(features_valid)

acc =  accuracy_score(target_valid, predictions_log)
result = f1_score(target_valid, predictions) 

probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)

In [17]:
print("Accuracy лучшей модели:", acc)
print("F1 для Логистической регрессии:", result)
print("AUC-ROC для Логистической регрессии:", auc_roc)

Accuracy лучшей модели: 0.8141836173721825
F1 для Логистической регрессии: 0.4822888283378747
AUC-ROC для Логистической регрессии: 0.6752914032301561


Ожидаемо, метрики логистической регрессии еще ниже.

In [18]:
#Модель случайного леса
best_model = None
best_result = 0
best_depth = 0
for est in range(10, 101, 10):
    for depth in range(1, 21, 1):
        model = RandomForestClassifier(random_state=12345, 
                                              n_estimators=est,
                                              max_depth=depth)
        model.fit(features_train, target_train)
        predictions_valid = model.predict(features_valid)
        result = f1_score(target_valid, predictions_valid)
        acc = accuracy_score(target_valid, predictions)
        if result > best_result:
            best_model = model
            best_est = est
            best_result = result
            best_depth = depth
            acc_best = acc
            probabilities_valid = best_model.predict_proba(features_valid)
            probabilities_one_valid = probabilities_valid[:, 1]
            auc_roc = roc_auc_score(target_valid, probabilities_one_valid)

In [19]:
print('Глубина',best_depth)
print('Кол-во деревьев', best_est)
print("Accuracy для случайного леса", acc_best)
print("F1 для случайного леса", best_result)
print("AUC-ROC для Леса:", auc_roc)

Глубина 20
Кол-во деревьев 100
Accuracy для случайного леса 0.7910940076965366
F1 для случайного леса 0.614065180102916
AUC-ROC для Леса: 0.8563237871932401


Странно, но факт: Модель решающего дерева имеет более высокие значение метрики F1, нежели случайный лес.

Наилучшей с точки зрения метрики F1 оказалась модель _решающего дерева_ с показателем 0.581. Также у данной модели самая высокая точность 0.8595.

Наилучшей с точки зрения AUC-ROC является модель _случайного леса_ с показателем 0.848.

Поскольку мы так и не получили целевое значение F1, то будем работать с дисбалансом.

## Дисбаланс

In [20]:
print(features_train[target_train == 0].shape)
features_train[target_train == 1].shape

(4359, 11)


(1095, 11)

Видно, что один класс преобладает, количество данных различается в 4 раза.

### Downsampling

In [21]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    
    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(features_downsampled, 
                                                       target_downsampled, 
                                                       random_state=12345)
    
    return features_downsampled, target_downsampled

features_downsampled, target_downsampled = downsample(features_train, 
                                                      target_train, 
                                                      1210/4790)

print (features_downsampled[target_downsampled == 0].shape)
print (features_downsampled[target_downsampled == 1].shape)

(1101, 11)
(1095, 11)


Сбалансировали классы. Теперь рассмотрим модель решающего дерева, она была лучшей по метрике F1.

In [22]:
#Посмотрим модель решающего дерева
best_model = None
best_result = 0
train_list = []
valid_list = []

for depth in range(1, 20):
    model = DecisionTreeClassifier(random_state=12345, 
                                   max_depth=depth) #модель с заданной глубиной дерева
    model.fit(features_downsampled, target_downsampled) # обучение модели
    predictions = model.predict(features_valid) #предсказания модели
    result = f1_score(target_valid, predictions) 
    acc = accuracy_score(target_valid, predictions)
    valid_list.append(result)
    if result > best_result:
        best_model = model
        best_result = result
        best_depth = depth
        best_acc = acc
        probabilities_valid = best_model.predict_proba(features_valid)
        probabilities_one_valid = probabilities_valid[:, 1]
        auc_roc = roc_auc_score(target_valid, probabilities_one_valid)

In [23]:
print("Наилучшая глубина: ", best_depth)
print("Accuracy лучшей модели:", best_acc)
print("F1 для Дерева решений:", best_result)
print("AUC-ROC для Дерева решени:", auc_roc)

Наилучшая глубина:  7
Accuracy лучшей модели: 0.7762506871907642
F1 для Дерева решений: 0.5782383419689119
AUC-ROC для Дерева решени: 0.8200453959787852


Показатели метрик модели упали, что можно связать с уменьшением кол-ва данных для обучения.

In [24]:
#Модель логистической регрессии
model_log = LogisticRegression(random_state=12345, 
                               solver='liblinear', 
                               max_iter=100)
model_log.fit(features_downsampled, target_downsampled)
predictions_log = model_log.predict(features_valid)

acc =  accuracy_score(target_valid, predictions_log)
result = f1_score(target_valid, predictions_log)

probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)

In [25]:
print("Accuracy лучшей модели:", acc)
print("F1 для Логистической регрессии:", result)
print("AUC-ROC для Логистической регрессии:", auc_roc)

Accuracy лучшей модели: 0.7328202308960967
F1 для Логистической регрессии: 0.5225933202357563
AUC-ROC для Логистической регрессии: 0.6991327195637191


В модели логистической регрессии матрика F1 значительно возросла и превышает целевой показатель.

In [26]:
#Модель случайного леса
best_model = None
best_result = 0
best_depth = 0
for est in range(10, 101, 10):
    for depth in range(1, 21, 1):
        model = RandomForestClassifier(random_state=12345, 
                                              n_estimators=est,
                                              max_depth=depth)
        model.fit(features_downsampled, target_downsampled)
        predictions_valid = model.predict(features_valid)
        result = f1_score(target_valid, predictions_valid)
        acc = accuracy_score(target_valid, predictions)
        if result > best_result:
            best_model = model
            best_est = est
            best_result = result
            best_depth = depth
            acc_best = acc
            probabilities_valid = best_model.predict_proba(features_valid)
            probabilities_one_valid = probabilities_valid[:, 1]
            auc_roc = roc_auc_score(target_valid, probabilities_one_valid)

In [27]:
print('Глубина',best_depth)
print('Кол-во деревьев', best_est)
print("Accuracy для случайного леса", acc_best)
print("F1 для случайного леса", best_result)
print("AUC-ROC для Леса:", auc_roc)

Глубина 8
Кол-во деревьев 100
Accuracy для случайного леса 0.6943375481033535
F1 для случайного леса 0.6205357142857143
AUC-ROC для Леса: 0.8663852515506548


В целом, после такого балансирования качество моделей не ухудшилось, но и сказать, что оно стало лучше - тоже нельзя.

### Upsampling

In [28]:
def upsample(features, target, repeat): 
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    
    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat) 
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    features_upsampled, target_upsampled = shuffle( features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

In [29]:
features_upsampled, target_upsampled = upsample(features_train,
                                                target_train, 
                                                4)

print (features_upsampled[target_upsampled == 0].shape)
print (features_upsampled[target_upsampled == 1].shape)

(4359, 11)
(4380, 11)


Увеличили меньшую выборку до уровня бОльшей.

Теперь повторим действия для моделей.

In [30]:
#Посмотрим модель решающего дерева
best_model = None
best_result = 0
train_list = []
valid_list = []

for depth in range(1, 20):
    model = DecisionTreeClassifier(random_state=12345, 
                                   max_depth=depth) #модель с заданной глубиной дерева
    model.fit(features_upsampled, target_upsampled) # обучение модели
    predictions = model.predict(features_valid) #предсказания модели
    result = f1_score(target_valid, predictions) 
    acc = accuracy_score(target_valid, predictions)
    valid_list.append(result)
    if result > best_result:
        best_model = model
        best_result = result
        best_depth = depth
        best_acc = acc
        probabilities_valid = best_model.predict_proba(features_valid)
        probabilities_one_valid = probabilities_valid[:, 1]
        auc_roc = roc_auc_score(target_valid, probabilities_one_valid)

In [31]:
print("Наилучшая глубина: ", best_depth)
print("Accuracy лучшей модели:", best_acc)
print("F1 для Дерева решений:", best_result)
print("AUC-ROC для Дерева решени:", auc_roc)

Наилучшая глубина:  5
Accuracy лучшей модели: 0.7713029136888401
F1 для Дерева решений: 0.574642126789366
AUC-ROC для Дерева решени: 0.8385951967159081


Странно, но факт - метрики упали по сравнению с несбалансированной выборкой.

In [32]:
#Модель логистической регрессии
model_log = LogisticRegression(random_state=12345, 
                               solver='liblinear', 
                               max_iter=100)
model_log.fit(features_upsampled, target_upsampled)
predictions_log = model_log.predict(features_valid)

acc =  accuracy_score(target_valid, predictions_log)
result = f1_score(target_valid, predictions_log)

probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)

In [33]:
print("Accuracy лучшей модели:", acc)
print("F1 для Логистической регрессии:", result)
print("AUC-ROC для Логистической регрессии:", auc_roc)

Accuracy лучшей модели: 0.719626168224299
F1 для Логистической регрессии: 0.4990176817288801
AUC-ROC для Логистической регрессии: 0.6978639169388428


Так же как и в случае даунсемплинга показатели выросли по отношению модели, обученной на исходных данных.

In [34]:
#Модель случайного леса
best_model = None
best_result = 0
best_depth = 0
for est in range(10, 101, 10):
    for depth in range(1, 21, 1):
        model = RandomForestClassifier(random_state=12345, 
                                              n_estimators=est,
                                              max_depth=depth)
        model.fit(features_upsampled, target_upsampled)
        predictions_valid = model.predict(features_valid)
        result = f1_score(target_valid, predictions_valid)
        acc = accuracy_score(target_valid, predictions)
        if result > best_result:
            best_model = model
            best_est = est
            best_result = result
            best_depth = depth
            acc_best = acc
            probabilities_valid = best_model.predict_proba(features_valid)
            probabilities_one_valid = probabilities_valid[:, 1]
            auc_roc = roc_auc_score(target_valid, probabilities_one_valid)

In [35]:
print('Глубина',best_depth)
print('Кол-во деревьев', best_est)
print("Accuracy для случайного леса", acc_best)
print("F1 для случайного леса", best_result)
print("AUC-ROC для Леса:", auc_roc)

Глубина 12
Кол-во деревьев 60
Accuracy для случайного леса 0.8037383177570093
F1 для случайного леса 0.6370967741935484
AUC-ROC для Леса: 0.8630929194258831


Подводя итог после работы с дисбалансом можно сказать, что наибольший показатель метрики F1 имеет модель Случайного леса. Так же нашли оптимальные гиперпараметры для нашего случая.

## Тестирование модели

Поскольку наиболее высокий результат показала модель Случайного леса, то будем рассматривать ее.

In [36]:
#Модель случайного леса
model = RandomForestClassifier(random_state=12345, 
                               n_estimators=100,
                               max_depth=12, 
                               class_weight='balanced')
model.fit(features_train, target_train)

predict = model.predict(features_test)
result = f1_score(target_test, predict)
acc = accuracy_score(target_test, predict)

probabilities_test = model.predict_proba(features_test)
probabilities_one_test = probabilities_test[:, 1]
auc_roc = roc_auc_score(target_test, probabilities_one_test)

In [37]:
print("Accuracy:", acc)
print("F1 для Логистической регрессии:", result)
print("AUC-ROC для Логистической регрессии:", auc_roc)

Accuracy: 0.8536853685368537
F1 для Логистической регрессии: 0.6253521126760563
AUC-ROC для Логистической регрессии: 0.8636651211473285


## Вывод

Была произведена предобработка данных, удалены пропуски. Использована ОНЕ.

Далее данные были разбиты на 3 выборки - тренировочную, валидационную и тестовую.

Исследован баланс классов. Произведено обучение 3 моделей и исследованы их метрики без учете дисбаланса. В ходе обучения было принято решение, что наиболее точной является модель _случайного леса_. 

Преобразованы данные для обучения (downsampling, upsampling). Был учтен дисбаланс в обучении моделей, модели были переобучены. Качество моделей после переобучения возросло.

На тестовой выборке модель _случайного леса_ показала следующие <u>?качественные?</u> метрики:

- Accuracy: 0.8536853685368537
- F1 для Логистической регрессии: 0.6253521126760563
- AUC-ROC для Логистической регрессии: 0.8636651211473285

Таким образом мы добились уровня F1 0.625, что выше целевого показателя (0.59).